<a href="https://colab.research.google.com/github/jcestevezc/Machine-Learning-Big-Data/blob/master/Machine%20Learning%20on%20Spark/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning on Spark

<img src="https://github.com/jcestevezc/Machine-Learning-Big-Data/blob/master/Machine%20Learning%20on%20Spark/ASB.jpeg?raw=true" width="400"> 

## 0.1 Methodology [CRISP-DM](https://github.com/jcestevezc/Machine-Learning-Big-Data/blob/master/Machine%20Learning%20on%20Spark/CRISP_DM.pdf) and [ASUM-DM](http://gforge.icesi.edu.co/ASUM-DM_External/index.htm#cognos.external.asum-DM_Teaser/deliveryprocesses/ASUM-DM_8A5C87D5.html)

Cross-industry standard process for data mining, known as CRISP-DM,[1] is an open standard process model that describes common approaches used by data mining experts. It is the most widely-used analytics model.

<img src="https://github.com/jcestevezc/Machine-Learning-Big-Data/blob/master/Machine%20Learning%20on%20Spark/CRISP.png?raw=true" width="400">

In 2015, IBM released a new methodology called Analytics Solutions Unified Method for Data Mining/Predictive Analytics (also known as ASUM-DM) which refines and extends CRISP-DM. Include *project management, more detailed deployment and operational description*.

<img src="https://github.com/jcestevezc/Machine-Learning-Big-Data/blob/master/Machine%20Learning%20on%20Spark/ASUM.jpg?raw=true" width="610">




## 0.2. Enviroment preparation

In [1]:
# ! pip install pyspark

In [2]:
# Import the PySpark module
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import round
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [3]:
##Create SparkContext
sc = SparkContext.getOrCreate()

# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('test') \
                    .getOrCreate()

In [4]:
# What version of Spark?
print(spark.version)

3.0.0


## 1. Business Understanding

* Determine business objectives
* Assess situation
* Determine data mining goals
* Produce project plan


#**Datos de una aerolinea queremos determinar si el vuelo tendra retrasos o no**
Para este caso el negocio decidio un retraso corresponde a un delay mayor a 15 min

## 2. Data Understanding

### 2.1 Collect initial data

In [5]:
## Is not the best choose for large data sets
flights = spark.read.csv('/content/sample_data/flights.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA'
                        )

### 2.2 Describe data

In [6]:
flights.printSchema()

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: integer (nullable = true)



### 2.3 Explore data

In [7]:
# Get number of records
print("The data contain %d records." % flights.count())

The data contain 50000 records.


In [8]:
# View the first five records
flights.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



In [9]:
flights.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
mon,50000,5.2351,3.437758623534696,0,11
dom,50000,15.66196,8.772488135606777,1,31
dow,50000,2.95236,1.966033503314405,0,6
carrier,50000,None,None,AA,WN
flight,50000,2054.31344,2182.4715300582875,1,6941
org,50000,None,None,JFK,TUS
mile,50000,882.40112,701.232785607705,67,4243
depart,50000,14.130952600000064,4.694052286573998,0.25,23.98
duration,50000,151.76582,87.04507290261697,30,560


In [10]:
# Check column data types
flights.dtypes

[('mon', 'int'),
 ('dom', 'int'),
 ('dow', 'int'),
 ('carrier', 'string'),
 ('flight', 'int'),
 ('org', 'string'),
 ('mile', 'int'),
 ('depart', 'double'),
 ('duration', 'int'),
 ('delay', 'int')]

### 2.4 Verify data quality

Assement about the data quality, a report similar to pandas profiling could be very usefuld.

## 3. Data preparation

### 3.1 Select data

In [11]:
flights = flights.drop('distance','hour')
flights.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



### 3.2 Clean data

#### 3.2.1 Filtering out missing data

In [12]:
flights.filter('delay is null').count()

2978

In [13]:
flights = flights.filter('delay is not null')
flights.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



In [14]:
flights = flights.dropna()
flights.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



### 3.3 Construct data

Convert 'mile' to 'km' and drop 'mile' column

In [15]:
flights = flights.withColumn('km', round(flights.mile * 1.60934, 0)).drop('mile')
flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|
+---+---+---+-------+------+---+------+--------+-----+------+
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30| 509.0|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8| 542.0|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|1989.0|
|  5|  2|  1|     UA|   704|SFO|  7.98|     102|    2| 885.0|
|  7|  2|  6|     AA|   380|ORD| 10.83|     135|   54|1180.0|
+---+---+---+-------+------+---+------+--------+-----+------+
only showing top 5 rows



Create 'label' column indicating whether flight delayed (1) or not (0)

In [16]:
flights = flights.withColumn('label', (flights.delay >=15).cast('integer'))
flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|label|
+---+---+---+-------+------+---+------+--------+-----+------+-----+
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|1989.0|    0|
|  5|  2|  1|     UA|   704|SFO|  7.98|     102|    2| 885.0|    0|
|  7|  2|  6|     AA|   380|ORD| 10.83|     135|   54|1180.0|    1|
+---+---+---+-------+------+---+------+--------+-----+------+-----+
only showing top 5 rows



### 3.4 Integrate data

Is not necessary for this dataset

### 3.5 Format data

#### 3.5.1 Indexing categorical data

In [17]:
# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights)

# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)

In [18]:
flights_indexed.show(5)

+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------+-------+
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|1989.0|    0|        1.0|    0.0|
|  5|  2|  1|     UA|   704|SFO|  7.98|     102|    2| 885.0|    0|        0.0|    1.0|
|  7|  2|  6|     AA|   380|ORD| 10.83|     135|   54|1180.0|    1|        1.0|    0.0|
+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------+-------+
only showing top 5 rows



#### 3.5.2 Assembling columns

In [19]:
# Create an assembler object
assembler = VectorAssembler(
    inputCols=['mon', 
               'dom' , 
               'dow',
               'carrier_idx',
               'org_idx',
               'km',
               'depart',
               'duration'
              ], outputCol='features')

# Consolidate predictor columns
flights = assembler.transform(flights_indexed)

In [20]:
flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------+-------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|label|carrier_idx|org_idx|            features|
+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------+-------+--------------------+
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|[0.0,22.0,2.0,0.0...|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|[2.0,20.0,4.0,0.0...|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|1989.0|    0|        1.0|    0.0|[9.0,13.0,1.0,1.0...|
|  5|  2|  1|     UA|   704|SFO|  7.98|     102|    2| 885.0|    0|        0.0|    1.0|[5.0,2.0,1.0,0.0,...|
|  7|  2|  6|     AA|   380|ORD| 10.83|     135|   54|1180.0|    1|        1.0|    0.0|[7.0,2.0,6.0,1.0,...|
+---+---+---+-------+------+---+------+--------+-----+------+-----+-----------+-------+--------------------+
only showing top 5 

#### 3.5.3 Train/test split

In [21]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights.count()
print(training_ratio)

0.796967376972481


## 4. Modeling

### 4.1 Select modeling technique: Decision Tree

In [22]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|0    |1.0       |[0.35067887590779917,0.6493211240922008]|
|1    |1.0       |[0.47227191413237923,0.5277280858676208]|
|1    |1.0       |[0.35067887590779917,0.6493211240922008]|
|1    |0.0       |[0.5358239508700102,0.4641760491299898] |
|1    |0.0       |[0.5358239508700102,0.4641760491299898] |
+-----+----------+----------------------------------------+
only showing top 5 rows



### 4.2 Assess model

In [23]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP) 
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1248|
|    0|       0.0| 2465|
|    1|       1.0| 3633|
|    0|       1.0| 2201|
+-----+----------+-----+

0.6387346810516392
